In [0]:
-- Set the performance slider to Level 1 (Cost effective)

-- Connect to any database of your choice

In [0]:
-- Create the necessary tables: https://github.com/awslabs/amazon-redshift-utils/blob/master/src/CloudDataWarehouseBenchmark/Cloud-DWB-Derived-from-TPCH/10GB/ddl.sql

create table lineitem (
  l_orderkey int8 not null ,
  l_partkey int8 not null,
  l_suppkey int4 not null,
  l_linenumber int4 not null,
  l_quantity numeric(12,2) not null,
  l_extendedprice numeric(12,2) not null,
  l_discount numeric(12,2) not null,
  l_tax numeric(12,2) not null,
  l_returnflag char(1) not null,
  l_linestatus char(1) not null,
  l_shipdate date not null ,
  l_commitdate date not null,
  l_receiptdate date not null,
  l_shipinstruct char(25) not null,
  l_shipmode char(10) not null,
  l_comment varchar(44) not null,
  Primary Key(L_ORDERKEY, L_LINENUMBER)
) distkey(l_orderkey) sortkey(l_shipdate,l_orderkey)  ;

create table part (
  p_partkey int8 not null ,
  p_name varchar(55) not null,
  p_mfgr char(25) not null,
  p_brand char(10) not null,
  p_type varchar(25) not null,
  p_size int4 not null,
  p_container char(10) not null,
  p_retailprice numeric(12,2) not null,
  p_comment varchar(23) not null,
  PRIMARY KEY (P_PARTKEY)
) distkey(p_partkey) sortkey(p_partkey);

In [0]:
-- Load data into the tables

copy lineitem from 's3://redshift-downloads/TPC-H/2.18/10GB/lineitem.tbl' iam_role default delimiter '|' region 'us-east-1';
copy part from 's3://redshift-downloads/TPC-H/2.18/10GB/part.tbl' iam_role default delimiter '|' region 'us-east-1';

In [0]:
-- Verify if the tables are loaded correctly

select count(*) from lineitem;  -- 59986052
select count(*) from part;  -- 2000000

In [0]:
-- Below is a procedure that will mimic a small workload with 2000 queries. We will use this to create a longer workload that spans over 24 hours. In order to do that we will use the "Schedule Queries" feature of the Redshift Query Editor.
-- Please schedule this query below to run at a frequency of 1 hour for next 24 hours. See README for more details.

CREATE OR REPLACE PROCEDURE loop_example_hourly()
LANGUAGE plpgsql
AS $$
DECLARE
    hour_count INT;
    i INT;
    promo_revenue NUMERIC;
BEGIN
    FOR i IN 1..5000 LOOP
        EXECUTE '
            SELECT /* TPC-H Q14 */ 100.00 * SUM(CASE
                    WHEN p_type LIKE ''PROMO%''
                        THEN l_extendedprice * (1 - l_discount)
                    ELSE 0
                END) / NULLIF(SUM(l_extendedprice * (1 - l_discount)), 0) AS promo_revenue
            FROM lineitem, part
            WHERE l_partkey = p_partkey
                AND l_shipdate >= DATE ''1995-09-01''
                AND l_shipdate < DATEADD(month, 1, CAST(''1995-09-01'' AS DATE));'
        INTO promo_revenue;

        RAISE NOTICE 'Iteration: %, Promo Revenue: %', i, promo_revenue;
    END LOOP;
END;
$$;

-- Disable the result cache
SET enable_result_cache_for_session TO off;

-- Run the workload
CALL loop_example_hourly();

In [0]:
-- Lets run the following query to observe how the compute_capacity decreases over time. It should start from 128 RPUs which is the default we started with. Over time the AI-based scaling and 
-- optimization will learn from the workload and perform necessary optimizations to favor "cost" as we have our slider position set to "Optimizes for cost", i.e., to the lowest setting of 1. 

select * from sys_serverless_usage
where start_time >= (select start_time from sys_query_history where query_text like '%TPC-H Q14%' order by start_time limit 1)
and end_time <= DATEADD(minute,1,(select end_time from sys_query_history where query_text like '%TPC-H Q14%' and end_time is not NULL order by end_time desc limit 1))
order by end_time asc;